In [9]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, jsonify
from flask import json


In [11]:
app = Flask(__name__)


In [12]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies1.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings1.csv')
ratings_df


,userId,movieId,rating
0,1,5,3
1,1,6,4
2,1,10,2
3,1,7,1
4,2,3,5
5,2,2,3
6,2,1,4
7,2,6,3
8,3,7,2
9,3,8,4


In [13]:
#     data = request.get_json(force=True)
#     title.append(data['title0'])
#     rating.append(int(data['rating0'])

#     title.append(data['title1'])
#     rating.append(int(data['rating1'])
    
#     title.append(data['title2'])
#     rating.append(int(data['rating2'])
    
#     title.append(data['title3'])
#     rating.append(int(data['rating3'])
   
#     title.append(data['title4'])
#     rating.append(int(data['rating4'])

In [14]:
title = [None]*5
rating = [None]*5

In [15]:

@app.route('/', methods=['POST','GET'])
def user_based():
    #Filtering out the movies by title
    data = request.get_json(force=True)
    title[0] = data['title0'] 
    rating[0] = int(data['rating0'])
    
    title[1] = data['title1'] 
    rating[1] = int(data['rating1'])
    
    title[2] = data['title2'] 
    rating[2] = int(data['rating2'])
    
    
    title[3] = data['title3'] 
    rating[3] = int(data['rating3'])

    title[4] = data['title4'] 
    rating[4] = int(data['rating4'])

    
    
    
    inputMovies = pd.DataFrame()

    inputMovies['rating'] = rating
    
    inputMovies['title'] = title
    
    
    
    
    
    
    
    
    inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
    #Then merging it so we can get the movieId. It's implicitly merging it by title.
    inputMovies = pd.merge(inputId, inputMovies)
    #Dropping information we won't use from the input dataframe
    #Final input dataframe
    #If a movie you added in above isn't here, then it might not be in the original 
    #dataframe or it might spelled differently, please check capitalisation.
    userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
    userSubsetGroup = userSubset.groupby(['userId'])
    userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)


        #Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
    pearsonCorrelationDict = {}

    #For every user group in our subset
    #Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient

    #For every user group in our subset
    for name, group in userSubsetGroup:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='movieId')
        inputMovies = inputMovies.sort_values(by='movieId')
        #Get the N for the formula
        nRatings = len(group)
        #Get the review scores for the movies that they both have in common
        temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp_df['rating'].tolist()
        #Let's also put the current user group reviews in a list format
        tempGroupList = group['rating'].tolist()
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorrelationDict[name] = 0


    pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
    pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['userId'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)
    topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
    l = []
    l = topUsersRating['userId']
    similar_user = l[0]
    topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
    tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
    tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']

    recommendation_df = pd.DataFrame()
#Now we take the weighted average
    recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
    recommendation_df['movieId'] = tempTopUsersRating.index
    recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
    x = recommendation_df.iloc[:,1]
    x = list(x)


    return jsonify([{"similar_user":str(l[0]),"c1":str(x[0]),"c2":str(x[1]),"c3":str(x[2]),"c4":str(x[3]),"c5":str(x[4]),"c6":str(x[5]),"c7":str(x[6])}])
    

In [16]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2019 06:50:54] "POST / HTTP/1.1" 200 -
